In [53]:
import gymnasium as gym

import stable_baselines3 as sb
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

In [ ]:
env = gym.make("LunarLander-v2")

print("_____OBSERVATION SPACE_____ \n")
# x, y, xDot, yDot, th, thDot, LeftLeg_on_ground?, RightLeg_on_ground?
print("Observation Space Shape", env.observation_space.shape)

print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)

_____OBSERVATION SPACE_____ 

Observation Space Shape (8,)

 _____ACTION SPACE_____ 

Action Space Shape 4


In [ ]:
save_dir = "/tmp/gym/"

env = make_vec_env("LunarLander-v2", n_envs=16)
render_env =  Monitor(gym.make("LunarLander-v2", render_mode="human"))
eval_env = Monitor(gym.make("LunarLander-v2"))

# Instantiate the agent
# model = sb.PPO('MlpPolicy', env)
# model = sb.DQN('MlpPolicy', env)
model = sb.PPO.load(save_dir + "/LunarLanderV2_1E6_16envs", env)

In [ ]:
model = sb.PPO(
    policy="MlpPolicy",
    env=env,
    n_steps=1024,
    batch_size=64,
    n_epochs=4,
    gamma=0.999,
    gae_lambda=0.98,
    ent_coef=0.01,
    verbose=1,
)

In [89]:
# Train the agent
model.learn(total_timesteps=int(2e6))

In [114]:
obs, _ = render_env.reset() 

for i in range(1500):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = render_env.step(action)
    # render_env.render()
    if terminated or truncated:
        # Note that the VecEnv resets automatically
        # when a done signal is encountered
        print(i+1, info)
        break
print(i+1, info)

172 {'episode': {'r': 312.766781, 'l': 172, 't': 2596.427161}}
172 {'episode': {'r': 312.766781, 'l': 172, 't': 2596.427161}}


In [104]:
eval_env = Monitor(gym.make("LunarLander-v2"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=100, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=284.79 +/- 20.162857401846193


In [98]:
save_dir = "/tmp/gym/"
model.save(save_dir + "/LunarLanderV2_3E6_16envs")